In [ ]:
import sys
from pathlib import Path
from os import getcwd, remove, walk
from os.path import join, basename
sys.path.append(str(Path(getcwd()).parent))

import numpy as np
import pandas as pd
from epiweeks import Week
from glob import glob
from datetime import date, timedelta, datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

from bulletin import default_input, default_output, tables_path, hoje
from bulletin.systems import Notifica, CasosConfirmados, eSUS

default_input = join(default_input)
default_output = join(default_output)

tables = dict([(Path(x).stem,pd.read_csv(join(tables_path,x))) for x in glob(join(tables_path,"*.csv"))])
municipios = tables['municipios']

pdidx = pd.IndexSlice

In [ ]:
def macro_sort(column):
    """Sort function"""
    macros = ['Leste', 'Oeste', 'Noroeste', 'Norte', 'Fora']
    correspondence = {macro: order for order, macro in enumerate(macros)}
    return column.map(correspondence)


In [ ]:
cc = CasosConfirmados()
cc.load(f"cc_{hoje.strftime('%d_%m_%Y')}", compress=True)

In [ ]:
faixa_etaria = tables['faixa_etaria_populacao']
faixa_etaria_municipio = tables['populacao_por_municipio_faixa_etaria'].set_index(['ibge','Município'])

faixa_etaria = faixa_etaria.loc[:15].append(pd.DataFrame([np.NaN,'80+',faixa_etaria.loc[16:,'populacao'].sum()], columns=[16],index=faixa_etaria.columns).T)
faixa_etaria.insert(3,'label',faixa_etaria_municipio.columns)
faixa_etaria.T

In [ ]:
pop_macro_rs_ibge = tables['populacao_por_municipio_faixa_etaria'].copy().join(municipios[['macro','rs','ibge','municipio']].set_index('ibge'), on='ibge').set_index(['macro','rs','ibge','municipio']).sort_index()
del pop_macro_rs_ibge['Município']
pop_macro_rs_ibge.columns = range(0,len(pop_macro_rs_ibge.columns))
pop_macro_rs_ibge

In [ ]:
casos = cc.df.join(municipios.set_index('ibge'), on='ibge_residencia')
casos.loc[casos['uf']!='PR','ibge_residencia'] = 999999
casos.update(casos[['ibge_residencia']].join(municipios.set_index('ibge'), on='ibge_residencia'))

casos['faixa_etaria'] = np.digitize(casos['idade'],faixa_etaria['bins'].dropna(),right=False)

casos['ano_caso'] = casos.loc[casos['data_diagnostico'].notna(),'data_diagnostico'].apply(lambda x: Week.fromdate(x).year)
casos['se_caso'] = casos.loc[casos['data_diagnostico'].notna(),'data_diagnostico'].apply(lambda x: Week.fromdate(x).week)

obitos = casos.loc[(casos['evolucao']==2)].copy()
obitos['ano_obito'] = obitos.loc[obitos['data_cura_obito'].notna(),'data_cura_obito'].apply(lambda x: Week.fromdate(x).year)
obitos['se_obito'] = obitos.loc[obitos['data_cura_obito'].notna(),'data_cura_obito'].apply(lambda x: Week.fromdate(x).week)

In [ ]:
casos_se_mun = casos.groupby(['ano_caso','se_caso','faixa_etaria','ibge_residencia'])[['identificacao']].count().rename(columns={'identificacao':'casos'}).unstack(['ibge_residencia','faixa_etaria']).fillna(0).astype(int)
casos_se_mun.index.names = ['ano','se']
casos_se_mun.columns = casos_se_mun.columns.set_names('quant',level=0)
casos_se_mun = casos_se_mun.reorder_levels(['ibge_residencia','faixa_etaria','quant'],axis=1).sort_index(level=0, axis=1)
casos_se_mun

In [ ]:
obitos_se_mun = obitos.groupby(['ano_caso','se_caso','faixa_etaria','ibge_residencia'])[['identificacao']].count().rename(columns={'identificacao':'obitos'}).unstack(['ibge_residencia','faixa_etaria']).fillna(0).astype(int)
obitos_se_mun.index.names = ['ano','se']
obitos_se_mun.columns = obitos_se_mun.columns.set_names('quant',level=0)
obitos_se_mun = obitos_se_mun.reorder_levels(['ibge_residencia','faixa_etaria','quant'],axis=1).sort_index(level=0, axis=1)
obitos_se_mun

In [ ]:
casos_obitos_se_mun = casos_se_mun.join(obitos_se_mun).reorder_levels(['ibge_residencia','faixa_etaria','quant'],axis=1).sort_index(level=0, axis=1).fillna(0).astype(int)
casos_obitos_se_mun

In [ ]:
casos_obitos_se_mun = casos_obitos_se_mun.reindex(pd.MultiIndex.from_product(casos_obitos_se_mun.columns.levels), axis=1).fillna(0).astype(int)
casos_obitos_se_mun

In [ ]:
casos_obitos_se_mun.T.to_excel(join(default_output,'ibge_faixa_quant_ano_se.xlsx'))